In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_score, classification_report, confusion_matrix, accuracy_score, recall_score, f1_score
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp
from sklearn.decomposition import PCA
from sklearn.linear_model import Perceptron
from sklearn.calibration import CalibratedClassifierCV
import xlsxwriter
import pandas as pd
from IPython.display import display
from sklearn.model_selection import StratifiedKFold




#Traitement du fichier .mat
cancercell = sp.loadmat('breastw.mat')
X = cancercell['X']
y = cancercell['y']
#print(cancercell)
y=np.reshape(y,(683,))
#fin traitement mat

In [21]:
penalty = ['l1', 'l2','elasticnet','None']
Max_iter = []
for k in range (1,15):
    Max_iter.append(50*k)
Eta =[]
for k in range (-5,3):
    Eta.append(10**(-k)) 
Alpha = []
for k in range (0,5):
    Alpha.append(10**(-k))

#data = np.array([['penalty','Max_iter','Eta','Alpha','F1 score','recall','precision','accuracy','AUC',]])

skf = StratifiedKFold(n_splits=10,shuffle=True) # équilibrage dataset

for train_index, test_index in skf.split(X,y):
    
    test =False
    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    y_train=np.ravel(y_train)
    y_test=np.ravel(y_test)

    #Méthode Perceptron simple

    test =False
    
    for k in penalty :
        for i in Max_iter :
            for j in Eta :
                for p in Alpha : 
                    #algo Perceptron
                    model_Percep = Perceptron(penalty=k, max_iter=i, eta0=j, alpha=p)
                    model_Percep.fit(X_train,y_train)
                    y_pred_percep = model_Percep.predict(X_test)
                    
                    #fin algo Perceptron
                    
                    #Scores 
                    F1_Percep = round(f1_score(y_test,y_pred_percep),3)
                    recall_Percep = round(recall_score(y_test,y_pred_percep),3)
                    precision_Percep = round(precision_score(y_test,y_pred_percep,zero_division=0),3)
                    accuracy_Percep = round(accuracy_score(y_test,y_pred_percep),3)
                    
                    #Construction matrice hyperparametres et scores
                    if test == False :
                        matrice_Percep = np.array([[k,i,j,p,F1_Percep,recall_Percep,precision_Percep,accuracy_Percep]])
                        test = True
                    else :
                        matrice_Percep = np.append(matrice_Percep,np.array([k,i,j,p,F1_Percep,recall_Percep,precision_Percep,accuracy_Percep]).reshape(1,8),axis=0)


#Dataframe 

df = pd.DataFrame(matrice_Percep, columns=['penalty','Max_iter','Eta','Alpha','F1 score','Recall','Precision','Accuracy'])
df = df.astype({'F1 score': float, 'Recall': float,'Precision': float,'Accuracy': float})
filtered_values_1 = np.where((df['F1 score']>=0.96) & (df['Recall']>=0.925) & (df['Precision']>=0.925))
display(df.iloc[filtered_values_1].sort_values(by=['F1 score'], ascending=False))


,penalty,Max_iter,Eta,Alpha,F1 score,Recall,Precision,Accuracy


In [30]:
filtered_values_2 = np.where((df['F1 score']>=0.96))
pd.options.display.max_rows = None
display(df.iloc[filtered_values_2].sort_values(by=['F1 score'], ascending=False))


,penalty,Max_iter,Eta,Alpha,F1 score,Recall,Precision,Accuracy
1156,elasticnet,50,0.01,0.1,0.96,1.0,0.923,0.971
1196,elasticnet,100,0.01,0.1,0.96,1.0,0.923,0.971
1236,elasticnet,150,0.01,0.1,0.96,1.0,0.923,0.971
1276,elasticnet,200,0.01,0.1,0.96,1.0,0.923,0.971
1316,elasticnet,250,0.01,0.1,0.96,1.0,0.923,0.971
1356,elasticnet,300,0.01,0.1,0.96,1.0,0.923,0.971
1396,elasticnet,350,0.01,0.1,0.96,1.0,0.923,0.971
1436,elasticnet,400,0.01,0.1,0.96,1.0,0.923,0.971
1476,elasticnet,450,0.01,0.1,0.96,1.0,0.923,0.971
1516,elasticnet,500,0.01,0.1,0.96,1.0,0.923,0.971


In [48]:
# evaluate label propagation on the semi-supervised learning dataset
from scipy.stats import f_oneway

dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']
Max_iter2 = [50, 100, 150, 250, 350, 450, 550, 650, 700]
skf2 = StratifiedKFold(n_splits=10) 
fold=0


algo = 0
for i in Max_iter2:
        test=False
        for train_index, test_index in skf2.split(X, y):
                fold=fold+1
                
                X_train, X_test = X[train_index,:], X[test_index,:]
                y_train, y_test = y[train_index], y[test_index]
                        
                model_Percep = Perceptron(penalty='elasticnet', max_iter=i, eta0=0.01, alpha=0.1)
                model_Percep.fit(X_train,y_train)
                y_pred_percep = model_Percep.predict(X_test)
                        
                F1=round(f1_score(y_test,y_pred_percep),3)
                    
                
                if test==False :
                        data = np.array([[F1]])
                        test=True
                else :
                        data = np.append(data,np.array([F1]).reshape(1,1),axis=0)
        if algo == 0  :
                df2=pd.DataFrame(data,columns=['[{}]'.format(i)])
                        
        else :
                df=pd.DataFrame(data,columns=['[{}]'.format(i)])
                df2=pd.concat([df2, df], axis = 1)
        algo = algo + 1
        
display(df2)
 

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/

,[50],[100],[150],[250],[350],[450],[550],[650],[700]
0,0.923,0.923,0.923,0.923,0.923,0.923,0.923,0.923,0.923
1,0.769,0.769,0.769,0.769,0.769,0.769,0.769,0.769,0.769
2,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500
3,0.917,0.917,0.917,0.917,0.917,0.917,0.917,0.917,0.917
4,0.923,0.923,0.923,0.923,0.923,0.923,0.923,0.923,0.923
5,0.958,0.958,0.958,0.958,0.958,0.958,0.958,0.958,0.958
6,0.452,0.452,0.452,0.452,0.452,0.452,0.452,0.452,0.452
7,0.814,0.814,0.814,0.814,0.814,0.814,0.814,0.814,0.814
8,0.873,0.873,0.873,0.873,0.873,0.873,0.873,0.873,0.873
9,0.960,0.960,0.960,0.960,0.960,0.960,0.960,0.960,0.960


In [49]:
t,p = f_oneway(df2['[50]'],df2['[100]'],df2['[150]'],df2['[250]'],df2['[350]'],df2['[450]'],df2['[550]'],df2['[650]'],df2['[700]'])
print('t statistic: %.3f' % t)
print('p value: %.13f' % p)

t statistic: 0.000
p value: 1.0000000000000
